In [1]:
!pip3 install langchain
!pip3 install accelerate
!pip3 install bitsandbytes
!pip install gpt4all 
!pip install transformers

!pip install numexpr
!pip install google-search-results


In [2]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate


from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification


import pandas as pd


/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Loading the data to pandas dataframe

In [3]:
data_path = "/home/naheed/Desktop/gpt4all/with_tools/archive/test.csv"
df = pd.read_csv(data_path)
df

,url,question,answers,Unnamed: 3,result_with_tools,result_without_tools,errors
0,http://www.freebase.com/view/en/jamaica,what does jamaican people speak?,['Jamaican Creole English Language' 'Jamaican ...,NaN,1.0,NaN,NaN
1,http://www.freebase.com/view/en/james_k_polk,what did james k polk do before he was president?,['Lawyer'],NaN,0.0,NaN,NaN
2,http://www.freebase.com/view/en/oregon_ducks,what is the oregon ducks 2012 football schedule?,['University of Oregon'],NaN,NaN,NaN,NaN
3,http://www.freebase.com/view/en/ken_barlow,who plays ken barlow in coronation street?,['Tony Warren'],NaN,NaN,NaN,NaN
4,http://www.freebase.com/view/en/chiune_sugihara,what happened after mr. sugihara died?,['Yaotsu'],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2027,http://www.freebase.com/view/en/david_beckham,what team did david beckham play for before la...,['Preston North End F.C.'],NaN,NaN,NaN,NaN
2028,http://www.freebase.com/view/en/france,who is the current leader of france 2010?,['Nicolas Sarkozy'],NaN,NaN,NaN,NaN
2029,http://www.freebase.com/view/en/knossos,where was the palace of knossos located?,['Crete' 'Greece'],NaN,NaN,NaN,NaN
2030,http://www.freebase.com/view/en/roswell_ufo_in...,where is roswell area 51?,['Roswell'],NaN,NaN,NaN,NaN


Adding local path for general llm and for evaluation

In [4]:
local_path = ("/home/naheed/Desktop/gpt4all/mistral-7b-openorca.Q4_0.gguf") #gpt4all
# local_path = ("/home/naheed/Desktop/gpt4all/ggml-model-q4_0.gguf") #TinyLlama/TinyLlama-1.1B-Chat-v0.6
# local_path = ("/home/naheed/Desktop/gpt4all/mistral-7b-instruct-v0.2.Q4_K_M.gguf") #mistral7b

eval_local_path = ("/home/naheed/Desktop/gpt4all/mistral-7b-openorca.Q4_0.gguf") #gpt4all
# eval_local_path = ("/home/naheed/Desktop/gpt4all/ggml-model-q4_0.gguf") #TinyLlama/TinyLlama-1.1B-Chat-v0.6
# eval_local_path = ("/home/naheed/Desktop/gpt4all/mistral-7b-instruct-v0.2.Q4_K_M.gguf") #mistral7b

In [5]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True , streaming=True)
eval_llm = GPT4All(model=eval_local_path, callbacks=callbacks, verbose=True , streaming=True)
# llm = GPT4All(model=local_path,device='gpu', callbacks=callbacks, verbose=False)

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
# llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

instantiating model for sentiment analysis

In [6]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
custom_promt = """
Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to answer questions about current news
Calculator: useful for when you need to answer questions about math

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

"""

In [8]:
import os
import pprint
%env SERPAPI_API_KEY = "e1f8cd9de27d3b78f9ab3428167883002e76c33b"
os.environ["SERPER_API_KEY"] = "e1f8cd9de27d3b78f9ab3428167883002e76c33b"

from langchain.utilities import GoogleSerperAPIWrapper
search = GoogleSerperAPIWrapper()


env: SERPAPI_API_KEY="e1f8cd9de27d3b78f9ab3428167883002e76c33b"


In [9]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.chains import LLMMathChain

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

llm_math_chain = LLMMathChain(llm=llm, verbose=True)

google_search = Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events",
        # description="useful for when you need to answer questions about current news",
        # coroutine= ... <- you can specify an async method if desired as well
    )

calculator =  Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        # coroutine= ... <- you can specify an async method if desired as well
    )

tools = [ google_search , calculator ]
# tools = [  calculator ]


# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [10]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)
    

class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:

        print("mmmmmmm"+llm_output+"kkkkkk")

        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
output_parser = CustomOutputParser()

In [11]:
# Set up the base template
template_asking_with_tools = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! 

New question: {question}
{agent_scratchpad}"""


template_for_comapring_answers = """Answer the following questions as best you can.


Use the following format:

Question: the input question you must answer
Final Answer: the final answer to the original input question


Promt1: "{llm_generated}" 
Promt2: "{actual_answer}"

Begin! 

Question: "{question}" Does Promt1 answer matches with Promt2 . Answer it in only Yes or No.
"""



prompt1_with_tools = CustomPromptTemplate(
    template=template_asking_with_tools,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["question", "intermediate_steps"]
)

prompt2_for_comparison = PromptTemplate(
    template=template_for_comapring_answers,
    input_variables=["question", "llm_generated" , "actual_answer"]
)



In [12]:
# for some warning
# pd.options.mode.chained_assignment = None  # default='warn'


# creating columns
df['result_with_tools']=""
df['result_without_tools']=""
df['errors']=""

start = 0 
finish = 100
df

for i in range(start,finish):
    
    print("Question "+ str(i) , end="....")

    question = df['question'][i]
    actual_answer = df['answers'][i]



    llm_chain1 = LLMChain(llm=llm, prompt=prompt1_with_tools)

    tool_names = [tool.name for tool in tools]
    agent = LLMSingleActionAgent(
        llm_chain=llm_chain1,
        output_parser=output_parser,
        stop=["\nObservation:"],
        allowed_tools=tool_names,
        tools = tools,
        agent = "zero-shot-react-description" ,
        verbose = True ,
        max_iterations = 6 ,
        # memory = memory ,
    
    )

    try:
        # agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory,handle_parsing_errors = True)
        agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True,handle_parsing_errors = True)
        llm_generated =  agent_executor.run(question = question)
    except:
        df.loc[i,'errors']=1

    print("==========================================================")


    try:
        llm_chain2 = LLMChain(llm=eval_llm, prompt=prompt2_for_comparison)
        final_result = llm_chain2.run(question = question , llm_generated = llm_generated , actual_answer = actual_answer)
    except:
        df.loc[i,'errors']=1



    classifier = pipeline("sentiment-analysis" , model = model,tokenizer=tokenizer)
    print("-----------------------sentiment------------------------------------")
    label = classifier([str(final_result)])
    print(label[0]["label"])


    # label[0]["label"]


    if(label[0]["label"] == "POSITIVE"):
        df.loc[i,'result_with_tools']= 1
        print("TRUE")
    else:
        df.loc[i,'result_with_tools']= 0
        print("FALSE")

    df.to_csv(data_path ,index = False)
    print("Data saved")



print("Process finished")
df = pd.read_csv(data_path , index = False)
df


Question 0....

> Entering new AgentExecutor chain...

Thought: I need to find out which language Jamaicans speak.
Action: Search
Action Input: "Jamaican language"
Observation: The official language of Jamaica is English, but many Jamaicans also speak Jamaican Patois or simply Patois.

Thought: Now that I know the languages spoken in Jamaica, I can provide an answer to the question.
Final Answer: Jamaican people speak English and some also speak Jamaican Patois.mmmmmmm
Thought: I need to find out which language Jamaicans speak.
Action: Search
Action Input: "Jamaican language"kkkkkk

Thought: I need to find out which language Jamaicans speak.
Action: Search
Action Input: "Jamaican language"

Observation:Jamaican Patois: Spoken language. Jamaican Patois is an English-based creole language with West African, Taíno, Irish, Spanish, Hindi, Portuguese, Chinese, and German influences, spoken primarily in Jamaica and among the Jamaican diaspora. Jamaican Patois Region: Americas. Jamaican Patoi

Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7ff66b3b55a0>
Traceback (most recent call last):
  File "/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 479, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 
